In [1]:
# Importamos librerías de análisis de datos
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
%matplotlib inline
from pywaffle import Waffle

In [2]:
# Cargamos el dataframe
df = pd.read_csv('train.csv', index_col='id', parse_dates=['fecha'])

In [3]:
# Reviso tipo de datos de columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240000 entries, 254099 to 121958
Data columns (total 22 columns):
titulo                        234613 non-null object
descripcion                   238381 non-null object
tipodepropiedad               239954 non-null object
direccion                     186928 non-null object
ciudad                        239628 non-null object
provincia                     239845 non-null object
antiguedad                    196445 non-null float64
habitaciones                  217529 non-null float64
garages                       202235 non-null float64
banos                         213779 non-null float64
metroscubiertos               222600 non-null float64
metrostotales                 188533 non-null float64
idzona                        211379 non-null float64
lat                           116512 non-null float64
lng                           116512 non-null float64
fecha                         240000 non-null datetime64[ns]
gimnasio           

In [4]:
# Busco limites numericos de los datos
columnas_de_interes = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'fecha', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas','centroscomercialescercanos']
for c in columnas_de_interes:
    print("{0} max: {1}".format(c, df[c].max()))
    print("{0} min: {1}".format(c, df[c].min()))


antiguedad max: 80.0
antiguedad min: 0.0
habitaciones max: 10.0
habitaciones min: 1.0
garages max: 3.0
garages min: 0.0
banos max: 4.0
banos min: 1.0
metroscubiertos max: 439.0
metroscubiertos min: 15.0
metrostotales max: 439.0
metrostotales min: 15.0
fecha max: 2016-12-31 00:00:00
fecha min: 2012-01-01 00:00:00
gimnasio max: 1.0
gimnasio min: 0.0
usosmultiples max: 1.0
usosmultiples min: 0.0
piscina max: 1.0
piscina min: 0.0
escuelascercanas max: 1.0
escuelascercanas min: 0.0
centroscomercialescercanos max: 1.0
centroscomercialescercanos min: 0.0


In [5]:
df.isnull().any()

titulo                         True
descripcion                    True
tipodepropiedad                True
direccion                      True
ciudad                         True
provincia                      True
antiguedad                     True
habitaciones                   True
garages                        True
banos                          True
metroscubiertos                True
metrostotales                  True
idzona                         True
lat                            True
lng                            True
fecha                         False
gimnasio                      False
usosmultiples                 False
piscina                       False
escuelascercanas              False
centroscomercialescercanos    False
precio                        False
dtype: bool

In [6]:
sininfobano = df[df['banos'].isnull() == True]
tiposdepropsininfobano = sininfobano.groupby('tipodepropiedad').agg({'tipodepropiedad': 'count'})
tiposdepropsininfobano

,tipodepropiedad
tipodepropiedad,
Apartamento,1337
Bodega comercial,660
Casa,8609
Casa en condominio,803
Casa uso de suelo,157
Departamento Compartido,7
Duplex,5
Edificio,1029
Garage,1


In [7]:
df.groupby('tipodepropiedad').agg({'banos': 'mean'})

,banos
tipodepropiedad,
Apartamento,1.816817
Bodega comercial,1.847185
Casa,2.240429
Casa en condominio,2.366065
Casa uso de suelo,2.404719
Departamento Compartido,1.791045
Duplex,1.360947
Edificio,2.626703
Garage,NaN


In [8]:
# Completo cantidad de banos para datos con NaN con valor promedio para cada tipodepropiedad
df['banos'] = df['banos'].fillna(df.groupby('tipodepropiedad')['banos'].transform(np.mean))

In [9]:
# Sigue habiendo NaN
df['banos'].isnull().any()

True

In [10]:
sininfobano = df[df['banos'].isnull() == True]
tiposdepropsininfobano = sininfobano.groupby('tipodepropiedad').agg({'tipodepropiedad': 'count'})
tiposdepropsininfobano

,tipodepropiedad
tipodepropiedad,
Garage,1
Hospedaje,1
Lote,5


In [11]:
df.groupby('tipodepropiedad').agg({'banos': 'mean'})

,banos
tipodepropiedad,
Apartamento,1.816817
Bodega comercial,1.847185
Casa,2.240429
Casa en condominio,2.366065
Casa uso de suelo,2.404719
Departamento Compartido,1.791045
Duplex,1.360947
Edificio,2.626703
Garage,NaN


In [12]:
df['banos'] = df['banos'].fillna(0)
df.groupby('tipodepropiedad').agg({'banos': 'mean'})

,banos
tipodepropiedad,
Apartamento,1.816817
Bodega comercial,1.847185
Casa,2.240429
Casa en condominio,2.366065
Casa uso de suelo,2.404719
Departamento Compartido,1.791045
Duplex,1.360947
Edificio,2.626703
Garage,0.000000


In [13]:
sininfogarage = df[df['garages'].isnull() == True]
tiposdepropsininfogarage = sininfogarage.groupby('tipodepropiedad').agg({'tipodepropiedad': 'count'})
tiposdepropsininfogarage

,tipodepropiedad
tipodepropiedad,
Apartamento,3736
Bodega comercial,525
Casa,23961
Casa en condominio,3237
Casa uso de suelo,240
Departamento Compartido,4
Duplex,7
Edificio,523
Huerta,5


In [14]:
df['garages'] = df['garages'].fillna(df.groupby('tipodepropiedad')['garages'].transform(np.mean))

In [39]:
df['garages'] = df['garages'].fillna(0)
sininfogarage = df[df['garages'].isnull() == True]
sininfogarage.shape

(0, 22)

In [35]:
df = df.astype({'garages': 'int32', 'banos': 'int32'})
df.dtypes

titulo                                object
descripcion                           object
tipodepropiedad                       object
direccion                             object
ciudad                                object
provincia                             object
antiguedad                           float64
habitaciones                         float64
garages                                int32
banos                                  int32
metroscubiertos                      float64
metrostotales                        float64
idzona                               float64
lat                                  float64
lng                                  float64
fecha                         datetime64[ns]
gimnasio                             float64
usosmultiples                        float64
piscina                              float64
escuelascercanas                     float64
centroscomercialescercanos           float64
precio                               float64
dtype: obj

In [19]:
df.groupby('tipodepropiedad').agg({'antiguedad': 'mean'})

,antiguedad
tipodepropiedad,
Apartamento,8.362021
Bodega comercial,9.635961
Casa,8.272588
Casa en condominio,6.328626
Casa uso de suelo,17.998377
Departamento Compartido,6.149254
Duplex,9.851632
Edificio,15.890924
Garage,NaN


In [28]:
df['antiguedad'].isnull().any()

False

In [21]:
sininfoantiguedad = df[df['antiguedad'].isnull() == True]
sininfoantiguedad.groupby('tipodepropiedad').agg({'tipodepropiedad':'count'})

,tipodepropiedad
tipodepropiedad,
Apartamento,9051
Bodega comercial,277
Casa,22691
Casa en condominio,3011
Casa uso de suelo,92
Departamento Compartido,7
Duplex,6
Edificio,195
Garage,1


In [22]:
df['antiguedad'] = df['antiguedad'].fillna(df.groupby('tipodepropiedad')['antiguedad'].transform(np.mean))
df['antiguedad'] = df['antiguedad'].fillna(0)

In [27]:
df['antiguedad'].isnull().any()

False

In [37]:
df = df.astype({'antiguedad': 'int32'})
df.dtypes

titulo                                object
descripcion                           object
tipodepropiedad                       object
direccion                             object
ciudad                                object
provincia                             object
antiguedad                             int32
habitaciones                         float64
garages                                int32
banos                                  int32
metroscubiertos                      float64
metrostotales                        float64
idzona                               float64
lat                                  float64
lng                                  float64
fecha                         datetime64[ns]
gimnasio                             float64
usosmultiples                        float64
piscina                              float64
escuelascercanas                     float64
centroscomercialescercanos           float64
precio                               float64
dtype: obj